This notebook runs a pipeline with the following steps:
* Create a test Visibility dataset.
   * No visibility sample noise. Just testing that things are working exactly as expected.
   * Using the GLEAM sky model and a common everybeam station beam model.
   * Add complex Gaussian noise corruptions to station Jones matrices.
   * Write to disk in MSv2 format.
* Read MSv2 data into Visibility dataset.$^{*}$
* Predict model visibilities (using GLEAM and everybeam).$^{*}$
* Do bandpass calibration.$^{*}$
* Check for convergence.$^{*}$

\* xarray dataset map_blocks() is used to distribute frequency sub-bands across dask tasks.

The pipeline is called once with a simple model, gain corruptions and a gain solver,
then again with a more complicated sky and beam model,
then again with gain and leakage corruptions and a polarised solver.

In [1]:
from time import perf_counter

from ska_sdp_instrumental_calibration.processing_tasks.lsm import Component
from ska_sdp_instrumental_calibration.workflow.pipelines import (
    bandpass_calibration,
)

<frozen abc>:106: FutureWarning: xarray subclass Visibility should explicitly define __slots__


In [2]:
gleamfile = "/data/EOS_1/mit183/gleamegc.dat"
eb_ms = "/data/EOS_1/mit183/SKA/SP-4626/OSKAR_MOCK.ms"
eb_coeffs = "/data/EOS_1/mit183/SKA/SP-4626/ska-sdp-func-everybeam/coeffs"

In [3]:
# Simple bandpass calibration of X and Y gains with a user-defined sky model
t0 = perf_counter()
bandpass_calibration.run(
    {
        "lsm": [
            Component(
                name="comp1", RAdeg=20.0, DEdeg=-30.0, flux=1.0, alpha=0.0
            ),
            Component(
                name="comp2", RAdeg=19.5, DEdeg=-29.9, flux=0.1, alpha=0.0
            ),
        ],
        "beam_type": "none",
        "ms_name": "demo.ms",
        "h5parm_name": "demo.h5",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2025-02-05 15:14:36,328 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2025-02-05 15:14:36,329 - workflow.pipeline_config - INFO - Predicting visibilities without a beam
2025-02-05 15:14:36,329 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2025-02-05 15:14:36,351 - workflow.utils - INFO - Using AA1-Low with 20 stations
2025-02-05 15:14:36,909 - processing_tasks.predict - INFO - No beam model used in predict
2025-02-05 15:14:37,081 - workflow.utils - INFO - Applying direction-independent gain corruptions
2025-02-05 15:14:37,542 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2025-02-05 15:14:37,543 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2025-02-05 15:14:38,615 - pipeline.bandpass_calibration - INFO - Setting calibration in 16-channel chunks
2025-02-05 15:14:38,617 - pipeline.bandpass_calibration - INFO - end_to_end_subbands = true
2025-02-05 15:14:38,885 

pipeline ran in 4.4 seconds


In [4]:
# Simple bandpass calibration of X and Y gains with GLEAM and EveryBeam
t0 = perf_counter()
bandpass_calibration.run(
    {
        # Required external data
        "gleamfile": gleamfile,
        "eb_ms": eb_ms,
        "eb_coeffs": eb_coeffs,
        # Other parameters
        "ms_name": "demo.ms",
        "h5parm_name": "demo.h5",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2025-02-05 15:14:40,767 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2025-02-05 15:14:40,768 - workflow.pipeline_config - INFO - Initialising the EveryBeam telescope model with /data/EOS_1/mit183/SKA/SP-4626/OSKAR_MOCK.ms
2025-02-05 15:14:40,769 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2025-02-05 15:14:40,770 - workflow.pipeline_config - INFO - Generating LSM for simulation with:
2025-02-05 15:14:40,771 - workflow.pipeline_config - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2025-02-05 15:14:40,771 - workflow.pipeline_config - INFO -  - Search radius: 5.0 deg
2025-02-05 15:14:40,772 - workflow.pipeline_config - INFO -  - Flux limit: 1 Jy
2025-02-05 15:14:41,358 - processing_tasks.lsm - INFO - extracted 37 GLEAM components
2025-02-05 15:14:41,359 - workflow.pipeline_config - INFO - LSM: found 37 components
2025-02-05 15:14:41,379 - workflow.utils - INFO - Using AA1-Low with 20 stations
2025-02-05 15:14:41,417 - pr

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2025-02-05 15:14:44,762 - workflow.utils - INFO - Applying direction-independent gain corruptions
2025-02-05 15:14:45,316 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2025-02-05 15:14:45,317 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2025-02-05 15:14:46,096 - pipeline.bandpass_calibration - INFO - Generating LSM for predict with:
2025-02-05 15:14:46,098 - pipeline.bandpass_calibration - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2025-02-05 15:14:46,099 - pipeline.bandpass_calibration - INFO -  - Search radius: 5.0 deg
2025-02-05 15:14:46,100 - pipeline.bandpass_calibration - INFO -  - Flux limit: 1 Jy
2025-02-05 15:14:46,777 - processing_tasks.lsm - INFO - extracted 37 GLEAM components
2025-02-05 15:14:46,778 - pipeline.bandpass_calibration - INFO - LSM: found 37 components
2025-02-05 15:14:46,779 - pipeline.bandpass_calibration - INFO - Setting calibration in 16-channel chunks
2025-02-05 15:

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2025-02-05 15:14:50,071 - pipeline.bandpass_calibration - INFO - Writing solutions to demo.h5
2025-02-05 15:14:50,077 - workflow.utils - INFO - exporting cal solutions to demo.h5
2025-02-05 15:14:50,088 - workflow.utils - INFO - output dimensions: {'time': 1, 'ant': 20, 'freq': 64, 'pol': 4}
2025-02-05 15:14:50,104 - pipeline.bandpass_calibration - INFO - Checking results
2025-02-05 15:14:50,106 - pipeline.bandpass_calibration - INFO - Convergence checks passed


pipeline ran in 9.7 seconds


In [5]:
# Bandpass calibration with leakage
t0 = perf_counter()
bandpass_calibration.run(
    {
        # Required external data
        "gleamfile": gleamfile,
        "eb_ms": eb_ms,
        "eb_coeffs": eb_coeffs,
        # Other parameters
        "ms_name": "demo.ms",
        "h5parm_name": "demo.h5",
        "gains": True,
        "leakage": True,
        "solver": "normal_equations",
    }
)
print(f"pipeline ran in {perf_counter() - t0:.1f} seconds")

2025-02-05 15:14:50,502 - workflow.pipeline_config - INFO - Simulating demo MSv2 input
2025-02-05 15:14:50,503 - workflow.pipeline_config - INFO - Initialising the EveryBeam telescope model with /data/EOS_1/mit183/SKA/SP-4626/OSKAR_MOCK.ms
2025-02-05 15:14:50,504 - workflow.pipeline_config - INFO - Generating a demo MSv2 Measurement Set demo.ms
2025-02-05 15:14:50,506 - workflow.pipeline_config - INFO - Generating LSM for simulation with:
2025-02-05 15:14:50,507 - workflow.pipeline_config - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2025-02-05 15:14:50,507 - workflow.pipeline_config - INFO -  - Search radius: 5.0 deg
2025-02-05 15:14:50,508 - workflow.pipeline_config - INFO -  - Flux limit: 1 Jy
2025-02-05 15:14:51,097 - processing_tasks.lsm - INFO - extracted 37 GLEAM components
2025-02-05 15:14:51,098 - workflow.pipeline_config - INFO - LSM: found 37 components
2025-02-05 15:14:51,107 - workflow.utils - INFO - Using AA1-Low with 20 stations
2025-02-05 15:14:51,155 - pr

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2025-02-05 15:14:54,510 - workflow.utils - INFO - Applying direction-independent gain corruptions
2025-02-05 15:14:54,511 - workflow.utils - INFO - Applying direction-independent leakage corruptions
2025-02-05 15:14:55,052 - pipeline.bandpass_calibration - INFO - Starting pipeline with 16-channel chunks
2025-02-05 15:14:55,053 - pipeline.bandpass_calibration - INFO - No dask cluster supplied. Using LocalCluster
2025-02-05 15:14:55,830 - pipeline.bandpass_calibration - INFO - Generating LSM for predict with:
2025-02-05 15:14:55,831 - pipeline.bandpass_calibration - INFO -  - Catalogue file: /data/EOS_1/mit183/gleamegc.dat
2025-02-05 15:14:55,832 - pipeline.bandpass_calibration - INFO -  - Search radius: 5.0 deg
2025-02-05 15:14:55,833 - pipeline.bandpass_calibration - INFO -  - Flux limit: 1 Jy
2025-02-05 15:14:56,541 - processing_tasks.lsm - INFO - extracted 37 GLEAM components
2025-02-05 15:14:56,542 - pipeline.bandpass_calibration - INFO - LSM: found 37 components
2025-02-05 15:14:56

Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest neighbor with frequency 110 MHz instead
Could not load dataset for frequency 100 MHz, using the nearest 

2025-02-05 15:15:00,639 - pipeline.bandpass_calibration - INFO - Writing solutions to demo.h5
2025-02-05 15:15:00,641 - workflow.utils - INFO - exporting cal solutions to demo.h5
2025-02-05 15:15:00,647 - workflow.utils - INFO - output dimensions: {'time': 1, 'ant': 20, 'freq': 64, 'pol': 4}
2025-02-05 15:15:00,661 - pipeline.bandpass_calibration - INFO - Checking results
2025-02-05 15:15:00,663 - pipeline.bandpass_calibration - INFO - Convergence checks passed


pipeline ran in 10.6 seconds
